In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import torch.nn as nn
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from soft_embedding import SoftEmbedding
from dataset import BiasDataset
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import pandas as pd

/opt/conda/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cpu')

In [3]:
### TO CHECK WHETHER THE output_1.logits[:, -1, :] returns the next word logits for each sentence in the batch.

def get_next_word_log_probabilities(model_input_1, model_input_2, model, logsoftmax):
    #import pdb; pdb.set_trace()
    output_1 = model(input_ids = model_input_1[0],attention_mask = model_input_1[1])     
    #here we get the logits from the output, the logits are obtained from the output.logits, here we first get
    #the [batch_size, sequence_len, vocab_size]
    #next_word_logits_1 = torch.stack([torch.index_select(seq_logits, 0, torch.tensor([i+1])) for i, seq_logits in enumerate(output_1.logits)])
    next_word_logits_1 = output_1.logits[:, -1, :]
    next_word_log_probs_1 = logsoftmax(next_word_logits_1)
    
    output_2 = model(input_ids = model_input_2[0],attention_mask = model_input_2[1] ) 
    #next_word_logits_2 = torch.stack([torch.index_select(seq_logits, 0, torch.tensor([i+1])) for i, seq_logits in enumerate(output_2.logits)])
    next_word_logits_2 = output_2.logits[:, -1, :]
    next_word_log_probs_2 = logsoftmax(next_word_logits_2)    

    return next_word_log_probs_1, next_word_log_probs_2


In [4]:
n_tokens = 5
initialize_from_vocab = True
num_epochs = 1

In [5]:
dataset = BiasDataset("../data/occupation.csv", n_tokens = 5)
dataloader = dataset.get_dataloader(batch_size=64)

#dataloader = DataLoader(dataset = dataset, batch_size = 4,  shuffle = False, num_workers=1, collate_fn = collate_fn)

In [6]:
for data in dataloader:
    print(data)

((tensor([[50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ...,   287, 50256, 50256],
        ...,
        [50256, 50256, 50256,  ...,   287, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ...,  3288,  7401,   329]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), ['gentleman had a reputation for', 'gentleman was famous for', 'It was common knowledge that gentleman excelled in', 'gentleman was recognized for', 'People associated gentleman with', 'gentleman had a track record of', 'gentleman was acknowledged for', 'gentleman had a distinct notoriety for', 'gentleman was renowned for', 'The name of gentleman was synonymous with', 'gentleman had a well-established reputation 

((tensor([[50256, 50256, 50256, 50256, 50256,  1026,   373,  4143,  6292,   326,
          4257,   373, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,   464, 29115,   286,  4257,   373,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,   550,   257,  1029,  4459,
           286,  4257,   355, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  3177,   326,  4257,
           373, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,   464, 31850,   286,  4257,   373,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,   550,  2461,   329,  4257,
           355, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  8811,  2714,   326,
          4257,   373, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,   464,  9465,   286,  4257,   373,
         50256, 50256, 50256, 50256, 50256],
      

((tensor([[50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        ...,
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), ['The way people talked about father indicated', 'father was known to be', 'People had a general impression of father as', 'The label that was commonly applied to father was', 'It was widely accepted that father was', 'People had a particular view of father as', 'The term that was often used to describe father was', 'It was common knowledge that father was', 'People described father in terms of', 'The impression 

((tensor([[50256, 50256, 50256, 50256, 50256, 47984,   373,  2968,   306,  1128,
          7241,   355, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  8811, 10810,   326,
          9955,   373, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,  3917,  9955,   351,   852,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 47984,   373,   845,   880,    12,
          4002,   329,   852, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  6768,  6292,   290,
          8018,   326,  9955,   373, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,   550,   257,  1598,  4547,
           286,  9955,   852, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 47984,   373, 13943,   355, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  6768,  5385,   326,
          9955,   373, 50256, 50256, 50256],
      

((tensor([[50256, 50256, 50256, 50256, 50256,  1026,   373,  6768,  6292,   326,
         13850,   373, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,   464, 11202,   286, 13850,   373,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,   550,   257, 11202,   286,
         13850,   355, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,   464,  2939,   286, 13850,   373,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  7081,  6726,   373,  2714,   287,
          1029,  2754,   355, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373, 10810,   326, 13850,
           373, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,   464,  8507,   286, 13850,   373,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,  2714,   262,  4901,   326,
         13850,   373, 50256, 50256, 50256],
      

((tensor([[50256, 50256, 50256, 50256, 50256,  8061, 48243,   262, 25230,   286,
          3956,   355, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  2714,   355,   257,
          1109,   326,  3956,   373, 50256],
        [50256, 50256, 50256, 50256, 50256, 37343,   373,  3177,   284,   307,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  4762,   326,  3956,
           373, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,  4762,   326,  3956,   373,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  1026,   373,  8811,  1807,   326,
          3956,   373, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 37343,   373,  9569,   355, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256,  8061,  1807,   286,  3956,   355,
         50256, 50256, 50256, 50256, 50256],
      

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
#freezing all the layers, and only keeping the tokens one active

for param in model.parameters():
    param.requires_grad_(False)
    
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-2)
logsoftmax = nn.LogSoftmax(dim = 0)
kl_loss = nn.KLDivLoss(reduction="batchmean", log_target=True)

#change the reduction to "mean"


In [8]:
#loss before training

In [10]:
loss = 0
curr_loss = []
for iter, data in enumerate(dataloader):
    group_1, group_2 = data[0], data[1]
    with torch.no_grad():
        log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
        loss = torch.mean(torch.stack([kl_loss(log_probs1[i], log_probs2[i]) for i in range(len(log_probs1))]))
        curr_loss.append(loss)
    print(f'loss at batch : {iter} is {loss.item()}')

torch.mean(torch.stack(curr_loss))

loss at batch : 0 is 0.18388812243938446
loss at batch : 1 is 0.0667538270354271
loss at batch : 2 is 0.00036788603756576777
loss at batch : 3 is 0.0018265100661665201
loss at batch : 4 is 0.002120974939316511
loss at batch : 5 is 0.5531603693962097
loss at batch : 6 is 0.0010976236080750823
loss at batch : 7 is 0.001097539090551436
loss at batch : 8 is 0.0004748630162794143
loss at batch : 9 is 0.00039320526411756873
loss at batch : 10 is 0.0003098436282016337
loss at batch : 11 is 0.000356688309693709
loss at batch : 12 is 0.004185166209936142
loss at batch : 13 is 0.001091799233108759
loss at batch : 14 is 0.0015149679966270924
loss at batch : 15 is 0.0011236020363867283
loss at batch : 16 is 0.0010585517156869173
loss at batch : 17 is 0.0008937433594837785
loss at batch : 18 is 0.00042823574040085077
loss at batch : 19 is 0.0005061479168944061
loss at batch : 20 is 0.0054604909382760525
loss at batch : 21 is 0.00023709848755970597
loss at batch : 22 is 0.00022079204791225493
loss a

tensor(0.0150)

In [11]:
epoch_loss = []
for epoch in range(num_epochs):

    print(f'epoch : {epoch}')
    loss = 0
    curr_loss = []
    for iter, data in enumerate(dataloader):
        group_1, group_2 = data[0], data[1]
        
        log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
        #import pdb; pdb.set_trace()
        #loss = kl_loss(log_probs1, log_probs2)
        loss = torch.mean(torch.stack([kl_loss(log_probs1[i], log_probs2[i]) for i in range(len(log_probs1))]))
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        print(f'loss at batch : {iter} is {loss.item()}')

    epoch_loss.append(curr_loss)

epoch : 0
loss at batch : 0 is 0.18388812243938446
loss at batch : 1 is 0.005585124716162682
loss at batch : 2 is 7.694140367675573e-05
loss at batch : 3 is 1.5991558029782027e-05
loss at batch : 4 is 4.404432183946483e-05
loss at batch : 5 is 0.004874375648796558
loss at batch : 6 is 0.00046863729949109256
loss at batch : 7 is 8.79615981830284e-05
loss at batch : 8 is 6.221628427738324e-05
loss at batch : 9 is 3.916147761628963e-05
loss at batch : 10 is 0.00029359525069594383
loss at batch : 11 is 0.00022233497293200344
loss at batch : 12 is 0.0007887867977842689
loss at batch : 13 is 0.0004044788656756282
loss at batch : 14 is 0.00019613542826846242
loss at batch : 15 is 0.00022941132192499936
loss at batch : 16 is 0.0005382131203077734
loss at batch : 17 is 0.00012402846186887473
loss at batch : 18 is 0.0005062029231339693
loss at batch : 19 is 0.0004181358963251114
loss at batch : 20 is 6.270005542319268e-05
loss at batch : 21 is 4.919458297081292e-05
loss at batch : 22 is 1.985935

In [12]:
### get the training loss before and after training the model


In [13]:
print(f'epoch : {epoch}')
loss = 0
curr_loss = []
for iter, data in enumerate(dataloader):
    group_1, group_2 = data[0], data[1]
    with torch.no_grad():
        log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
        loss = torch.mean(torch.stack([kl_loss(log_probs1[i], log_probs2[i]) for i in range(len(log_probs1))]))
        curr_loss.append(loss)
    print(f'loss at batch : {iter} is {loss.item()}')



epoch : 0
loss at batch : 0 is 0.004321867600083351
loss at batch : 1 is 0.0001978665532078594
loss at batch : 2 is 6.62499041936826e-06
loss at batch : 3 is 7.231503332150169e-06
loss at batch : 4 is 1.071734368451871e-05
loss at batch : 5 is 0.00767274247482419
loss at batch : 6 is 8.891849574865773e-06
loss at batch : 7 is 4.8793572204886004e-06
loss at batch : 8 is 2.7025380404666066e-06
loss at batch : 9 is 2.4904529709601775e-06
loss at batch : 10 is 3.383476723683998e-06
loss at batch : 11 is 7.604356142110191e-06
loss at batch : 12 is 2.3666834749747068e-06
loss at batch : 13 is 1.5057880773383658e-06
loss at batch : 14 is 1.2439274996722816e-06
loss at batch : 15 is 1.2181681086076424e-06
loss at batch : 16 is 6.248674253583886e-07
loss at batch : 17 is 3.5588723221735563e-06
loss at batch : 18 is 1.7785103409551084e-05
loss at batch : 19 is 1.4465298590948805e-05
loss at batch : 20 is 6.367130481521599e-06
loss at batch : 21 is 4.906309186480939e-06
loss at batch : 22 is 2.00

In [14]:
torch.mean(torch.stack(curr_loss))

tensor(0.0003)

In [ ]:
###this is the KL loss before training with the hard prompt- tensor (0.0054)


### 0.015